In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.preprocessing import LabelEncoder
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score

In [2]:
df_train = pd.read_csv('train.csv')

In [3]:
df_test=pd.read_csv('test.csv')


In [4]:
y_train=df_train['project_is_approved']

In [5]:
x_train=df_train.copy()
x_train.drop(columns=['project_is_approved'])

,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,total_quantity,total_price
0,22aa8e52e6268788a9e5c10932da6f54,Mrs.,NJ,2016-06-20 23:05:15,Grades 3-5,Literacy & Language,"Literacy, Literature & Writing",To Infinity and Beyond...Let's Edit!,"\""Can you be our teacher next year, you made S...",I would love for my students to have the oppor...,NaN,NaN,My students need a document camera to assist i...,0,1,264.99
1,59d70f9999d4b63a2c95c42d61d8468c,Ms.,GA,2016-09-01 08:11:20,Grades PreK-2,"Literacy & Language, Special Needs","Literacy, Special Needs",Reading With Our Ears?!,Living in poverty often means lack of access t...,Can you imagine a child not having the simple ...,NaN,NaN,My students need a listening center with headp...,12,5,847.00
2,168074f8c5e0bec129596844eab8c53d,Mr.,TX,2016-05-09 11:40:49,Grades 9-12,Applied Learning,Extracurricular,Debate Materials for Upcoming Year,We are a High School Speech and Debate Team. W...,My students are a mixture of male and female. ...,They will use the materials for basic function...,"We have to beg, borrow and steal materials fro...",My students need basic supplies for the upcomi...,1,13,222.20
3,2ac6e7a8b435d3fa6dc3c53a910de7e0,Ms.,CA,2016-11-27 18:15:15,Grades PreK-2,"Literacy & Language, Special Needs","Literacy, Special Needs",Help letter sounds come to life for my students!,My students are a fun bunch of diverse kids wh...,Alphabet and sound tubs will allow sounds to c...,NaN,NaN,My students need letter sound tubs to allow th...,0,5,318.97
4,6492fd8d2fe4961e207facd59b807c39,Ms.,LA,2016-07-07 19:20:49,Grades 3-5,Literacy & Language,Literacy,Using Technology to Develop Comprehension Skil...,My students are working very hard to increase ...,I want to began the school year with a place w...,NaN,NaN,My students need a large class rug to feel fre...,0,25,606.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87393,196b6ae042c9b7c7c292d6774e80685b,Ms.,VA,2016-09-22 09:04:26,Grades 9-12,Literacy & Language,Literacy,No Fear Romeo and Juliet,Many of my students are lacking basic school s...,My students will be reading Romeo and Juliet. ...,NaN,NaN,My students need copies of Romeo and Juliet so...,12,35,4.34
87394,1ebffc84876c362a8766fabfbe8c26da,Mrs.,AZ,2016-10-05 21:55:24,Grades PreK-2,Health & Sports,Health & Wellness,Wiggle and Read!,"I teach in a very rural, impoverished, Title 1...","My kids are constantly wiggly, jiggly and bala...",NaN,NaN,My students need alternative seating options i...,1,21,156.64
87395,a46a4a6eecc869b94fecfc5cb2b20348,Mrs.,MI,2016-08-21 17:58:54,Grades 3-5,Health & Sports,"Health & Wellness, Nutrition Education","Grow Green, Eat Green!",My students are fantastic kids that love to le...,My students told me that they want to start th...,NaN,NaN,My students need a Growlab II Mobile Indoor Ga...,23,3,707.37
87396,1a26b6896e5b6f847e41c3a8fb305e29,Ms.,CA,2016-09-21 14:28:28,Grades 3-5,"Literacy & Language, Special Needs","Literacy, Special Needs",We Are In Love With Mo Willems Books!,My students are second through fifth grade stu...,My students are great at reading. They also r...,NaN,NaN,My students need new high interest books such ...,16,16,136.63


In [6]:
df=pd.concat([x_train,df_test])
df

,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,total_quantity,total_price
0,22aa8e52e6268788a9e5c10932da6f54,Mrs.,NJ,2016-06-20 23:05:15,Grades 3-5,Literacy & Language,"Literacy, Literature & Writing",To Infinity and Beyond...Let's Edit!,"\""Can you be our teacher next year, you made S...",I would love for my students to have the oppor...,NaN,NaN,My students need a document camera to assist i...,0,1.0,1,264.99
1,59d70f9999d4b63a2c95c42d61d8468c,Ms.,GA,2016-09-01 08:11:20,Grades PreK-2,"Literacy & Language, Special Needs","Literacy, Special Needs",Reading With Our Ears?!,Living in poverty often means lack of access t...,Can you imagine a child not having the simple ...,NaN,NaN,My students need a listening center with headp...,12,1.0,5,847.00
2,168074f8c5e0bec129596844eab8c53d,Mr.,TX,2016-05-09 11:40:49,Grades 9-12,Applied Learning,Extracurricular,Debate Materials for Upcoming Year,We are a High School Speech and Debate Team. W...,My students are a mixture of male and female. ...,They will use the materials for basic function...,"We have to beg, borrow and steal materials fro...",My students need basic supplies for the upcomi...,1,1.0,13,222.20
3,2ac6e7a8b435d3fa6dc3c53a910de7e0,Ms.,CA,2016-11-27 18:15:15,Grades PreK-2,"Literacy & Language, Special Needs","Literacy, Special Needs",Help letter sounds come to life for my students!,My students are a fun bunch of diverse kids wh...,Alphabet and sound tubs will allow sounds to c...,NaN,NaN,My students need letter sound tubs to allow th...,0,1.0,5,318.97
4,6492fd8d2fe4961e207facd59b807c39,Ms.,LA,2016-07-07 19:20:49,Grades 3-5,Literacy & Language,Literacy,Using Technology to Develop Comprehension Skil...,My students are working very hard to increase ...,I want to began the school year with a place w...,NaN,NaN,My students need a large class rug to feel fre...,0,0.0,25,606.41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21845,7c34671b3e5d14402c3b43a1995b1994,Mrs.,AL,2017-03-25 21:13:41,Grades 9-12,Literacy & Language,Literacy,Students Learning to Be Teachers,My students are great and I am lucky. I teach ...,In my Early Child Development classes my high ...,NaN,NaN,"My students need felt, colored pencils, chart ...",125,NaN,12,105.96
21846,c5a34e13ad7942cf6b515ff7bd161f7c,Ms.,OK,2016-07-20 20:04:30,Grades 3-5,"Literacy & Language, Math & Science","Literature & Writing, Mathematics",Bouncin' Our Way to a Better Brain!,Did you know 85% of learners are kinesthetic? ...,The stability balls are to improve my students...,NaN,NaN,My students need stability balls to improve fo...,0,NaN,1,447.98
21847,ffd37d59c0e76bb6bb125a1b67424d7c,Ms.,CA,2017-02-21 13:52:12,Grades 3-5,Math & Science,"Applied Sciences, Mathematics",Let's Build a Video Game!,"My students come from all walks of life, their...",The students are learning how to use design th...,NaN,NaN,My students need 10 Bloxel Builder Sets and 5 ...,0,NaN,15,63.98
21848,198d7d0a27e4b11d95be7207b0e26163,Mrs.,MI,2016-11-21 12:12:02,Grades 9-12,Special Needs,Special Needs,Stop Fidgeting and Standup!,Our resource program supports 148 students wit...,"As a teacher, I never sit down. Why make my s...",NaN,NaN,My students need stand-up desks to help them f...,40,NaN,2,199.00


In [7]:
df1=df.reset_index(drop=True)

df1 = df1.astype(str)
df1.fillna('',inplace = True)
df1
values=df1.isnull().sum()

print(values)

teacher_id                                      0
teacher_prefix                                  0
school_state                                    0
project_submitted_datetime                      0
project_grade_category                          0
project_subject_categories                      0
project_subject_subcategories                   0
project_title                                   0
project_essay_1                                 0
project_essay_2                                 0
project_essay_3                                 0
project_essay_4                                 0
project_resource_summary                        0
teacher_number_of_previously_posted_projects    0
project_is_approved                             0
total_quantity                                  0
total_price                                     0
dtype: int64


In [8]:
df1=df1.drop(columns='project_submitted_datetime')
df1

,teacher_id,teacher_prefix,school_state,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,project_is_approved,total_quantity,total_price
0,22aa8e52e6268788a9e5c10932da6f54,Mrs.,NJ,Grades 3-5,Literacy & Language,"Literacy, Literature & Writing",To Infinity and Beyond...Let's Edit!,"\""Can you be our teacher next year, you made S...",I would love for my students to have the oppor...,nan,nan,My students need a document camera to assist i...,0,1.0,1,264.99
1,59d70f9999d4b63a2c95c42d61d8468c,Ms.,GA,Grades PreK-2,"Literacy & Language, Special Needs","Literacy, Special Needs",Reading With Our Ears?!,Living in poverty often means lack of access t...,Can you imagine a child not having the simple ...,nan,nan,My students need a listening center with headp...,12,1.0,5,847.0
2,168074f8c5e0bec129596844eab8c53d,Mr.,TX,Grades 9-12,Applied Learning,Extracurricular,Debate Materials for Upcoming Year,We are a High School Speech and Debate Team. W...,My students are a mixture of male and female. ...,They will use the materials for basic function...,"We have to beg, borrow and steal materials fro...",My students need basic supplies for the upcomi...,1,1.0,13,222.2
3,2ac6e7a8b435d3fa6dc3c53a910de7e0,Ms.,CA,Grades PreK-2,"Literacy & Language, Special Needs","Literacy, Special Needs",Help letter sounds come to life for my students!,My students are a fun bunch of diverse kids wh...,Alphabet and sound tubs will allow sounds to c...,nan,nan,My students need letter sound tubs to allow th...,0,1.0,5,318.96999999999997
4,6492fd8d2fe4961e207facd59b807c39,Ms.,LA,Grades 3-5,Literacy & Language,Literacy,Using Technology to Develop Comprehension Skil...,My students are working very hard to increase ...,I want to began the school year with a place w...,nan,nan,My students need a large class rug to feel fre...,0,0.0,25,606.4100000000001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109243,7c34671b3e5d14402c3b43a1995b1994,Mrs.,AL,Grades 9-12,Literacy & Language,Literacy,Students Learning to Be Teachers,My students are great and I am lucky. I teach ...,In my Early Child Development classes my high ...,nan,nan,"My students need felt, colored pencils, chart ...",125,nan,12,105.96
109244,c5a34e13ad7942cf6b515ff7bd161f7c,Ms.,OK,Grades 3-5,"Literacy & Language, Math & Science","Literature & Writing, Mathematics",Bouncin' Our Way to a Better Brain!,Did you know 85% of learners are kinesthetic? ...,The stability balls are to improve my students...,nan,nan,My students need stability balls to improve fo...,0,nan,1,447.98
109245,ffd37d59c0e76bb6bb125a1b67424d7c,Ms.,CA,Grades 3-5,Math & Science,"Applied Sciences, Mathematics",Let's Build a Video Game!,"My students come from all walks of life, their...",The students are learning how to use design th...,nan,nan,My students need 10 Bloxel Builder Sets and 5 ...,0,nan,15,63.98
109246,198d7d0a27e4b11d95be7207b0e26163,Mrs.,MI,Grades 9-12,Special Needs,Special Needs,Stop Fidgeting and Standup!,Our resource program supports 148 students wit...,"As a teacher, I never sit down. Why make my s...",nan,nan,My students need stand-up desks to help them f...,40,nan,2,199.0


In [9]:
df1['Merged']=df1['project_title']+df1['project_essay_1']+df1['project_essay_2']+df1['project_essay_3']+df1['project_essay_4']+df1['project_resource_summary']
df1.drop(columns=['project_title','project_essay_1','project_essay_2','project_essay_3','project_essay_4','project_resource_summary']
, inplace=True)

In [10]:
df1['teacher_prefix'] = df1['teacher_prefix'].astype(str)
df1

,teacher_id,teacher_prefix,school_state,project_grade_category,project_subject_categories,project_subject_subcategories,teacher_number_of_previously_posted_projects,project_is_approved,total_quantity,total_price,Merged
0,22aa8e52e6268788a9e5c10932da6f54,Mrs.,NJ,Grades 3-5,Literacy & Language,"Literacy, Literature & Writing",0,1.0,1,264.99,"To Infinity and Beyond...Let's Edit!\""Can you ..."
1,59d70f9999d4b63a2c95c42d61d8468c,Ms.,GA,Grades PreK-2,"Literacy & Language, Special Needs","Literacy, Special Needs",12,1.0,5,847.0,Reading With Our Ears?!Living in poverty often...
2,168074f8c5e0bec129596844eab8c53d,Mr.,TX,Grades 9-12,Applied Learning,Extracurricular,1,1.0,13,222.2,Debate Materials for Upcoming YearWe are a Hig...
3,2ac6e7a8b435d3fa6dc3c53a910de7e0,Ms.,CA,Grades PreK-2,"Literacy & Language, Special Needs","Literacy, Special Needs",0,1.0,5,318.96999999999997,Help letter sounds come to life for my student...
4,6492fd8d2fe4961e207facd59b807c39,Ms.,LA,Grades 3-5,Literacy & Language,Literacy,0,0.0,25,606.4100000000001,Using Technology to Develop Comprehension Skil...
...,...,...,...,...,...,...,...,...,...,...,...
109243,7c34671b3e5d14402c3b43a1995b1994,Mrs.,AL,Grades 9-12,Literacy & Language,Literacy,125,nan,12,105.96,Students Learning to Be TeachersMy students ar...
109244,c5a34e13ad7942cf6b515ff7bd161f7c,Ms.,OK,Grades 3-5,"Literacy & Language, Math & Science","Literature & Writing, Mathematics",0,nan,1,447.98,Bouncin' Our Way to a Better Brain!Did you kno...
109245,ffd37d59c0e76bb6bb125a1b67424d7c,Ms.,CA,Grades 3-5,Math & Science,"Applied Sciences, Mathematics",0,nan,15,63.98,Let's Build a Video Game!My students come from...
109246,198d7d0a27e4b11d95be7207b0e26163,Mrs.,MI,Grades 9-12,Special Needs,Special Needs,40,nan,2,199.0,Stop Fidgeting and Standup!Our resource progra...


In [11]:
columns_to_encode = ['teacher_id','teacher_prefix', 'project_grade_category', 'project_subject_categories','project_subject_subcategories','school_state']
label_encoder = LabelEncoder()
for column in columns_to_encode:
    df1[column] = label_encoder.fit_transform(df1[column])
df1

,teacher_id,teacher_prefix,school_state,project_grade_category,project_subject_categories,project_subject_subcategories,teacher_number_of_previously_posted_projects,project_is_approved,total_quantity,total_price,Merged
0,9656,2,31,0,24,318,0,1.0,1,264.99,"To Infinity and Beyond...Let's Edit!\""Can you ..."
1,25176,3,10,3,30,326,12,1.0,5,847.0,Reading With Our Ears?!Living in poverty often...
2,6294,1,43,2,0,209,1,1.0,13,222.2,Debate Materials for Upcoming YearWe are a Hig...
3,11919,3,4,3,30,326,0,1.0,5,318.96999999999997,Help letter sounds come to life for my student...
4,28247,3,18,0,24,317,0,0.0,25,606.4100000000001,Using Technology to Develop Comprehension Skil...
...,...,...,...,...,...,...,...,...,...,...,...
109243,34847,2,1,2,24,317,125,nan,12,105.96,Students Learning to Be TeachersMy students ar...
109244,55601,3,36,0,28,331,0,nan,1,447.98,Bouncin' Our Way to a Better Brain!Did you kno...
109245,72108,3,4,0,32,18,0,nan,15,63.98,Let's Build a Video Game!My students come from...
109246,7103,2,22,2,46,392,40,nan,2,199.0,Stop Fidgeting and Standup!Our resource progra...


In [12]:
df1['Merged'] = df1['Merged'].str.replace(r'\\n|\\r|\n|\r|\'|\\"', '', regex=True)

In [13]:

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
columns_to_scale = ['teacher_id','total_price','project_subject_subcategories']
scaled_columns = scaler.fit_transform(df1.loc[:, columns_to_scale])

scaled_df = pd.DataFrame(scaled_columns, columns=columns_to_scale)
df1[columns_to_scale] = scaled_df

In [14]:
sw = stopwords.words('english')
sw.extend(['','?','/'])  
def preprocess_text(text):
    words = text.lower().split()
    filtered_words = [word for word in words if word not in sw]
    processed_text = ' '.join(filtered_words)
    
    return processed_text

df1['Merged'] = df1['Merged'].apply(preprocess_text)

In [15]:
from nltk.stem import SnowballStemmer
sb = SnowballStemmer('english')

def stem_text(text):
    return sb.stem(text)
df1['Merged'] = df1['Merged'].apply(stem_text)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer(max_features=5000)

tfidf_matrix = tfidf_vectorizer.fit_transform(df1['Merged'])

tfidf_array = tfidf_matrix.toarray()
tfidf_df = pd.DataFrame(tfidf_array, columns=tfidf_vectorizer.get_feature_names_out())
df1 = pd.concat([df1, tfidf_df], axis=1)
df1.drop(columns=['Merged'], inplace=True)

In [17]:
split_row=87398
train1=df1.iloc[:split_row]
test1=df1.iloc[split_row:]
train1

,teacher_id,teacher_prefix,school_state,project_grade_category,project_subject_categories,project_subject_subcategories,teacher_number_of_previously_posted_projects,project_is_approved,total_quantity,total_price,...,youngest,your,youre,yourself,youth,youtube,zenergy,zip,zone,zoo
0,0.133801,2,31,0,24,0.7950,0,1.0,1,0.026437,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.18141,0.0
1,0.348858,3,10,3,30,0.8150,12,1.0,5,0.084648,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
2,0.087214,1,43,2,0,0.5225,1,1.0,13,0.022158,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
3,0.165159,3,4,3,30,0.8150,0,1.0,5,0.031836,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
4,0.391412,3,18,0,24,0.7925,0,0.0,25,0.060585,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87393,0.097843,3,45,2,24,0.7925,12,1.0,35,0.000368,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
87394,0.118295,2,3,3,8,0.7225,1,0.0,21,0.015601,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
87395,0.642427,2,22,0,8,0.7375,23,1.0,3,0.070683,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0
87396,0.100752,3,4,0,30,0.8150,16,1.0,16,0.013599,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0


In [18]:
test1
test1.isnull().sum()

teacher_id                    0
teacher_prefix                0
school_state                  0
project_grade_category        0
project_subject_categories    0
                             ..
youtube                       0
zenergy                       0
zip                           0
zone                          0
zoo                           0
Length: 5010, dtype: int64

In [19]:
X = train1 
Y = y_train

In [20]:
from imblearn.over_sampling import SMOTE


smote = SMOTE(random_state=11,sampling_strategy='minority')

X_resampled, Y_resampled = smote.fit_resample(X, Y)


In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score

rf_model = RandomForestClassifier(n_estimators=100, random_state=11)
rf_model.fit(X_resampled, Y_resampled)
y_pred = rf_model.predict(X_resampled)


In [22]:

predictions = rf_model.predict(test1)

print(predictions)

# Create a DataFrame for predictions
predictions_df = pd.DataFrame({'predicted_label': predictions})

# Save the predictions to a CSV file without indexes
predictions_df.to_csv('predict_se22uari175.csv', index=False)

# Print the shape of the predictions DataFrame
print(predictions_df.shape)

[1 1 1 ... 1 1 1]
(21850, 1)


In [24]:
# Count the number of 0's and 1's in predictions
counts = predictions_df['predicted_label'].value_counts()

# Print the counts
print("Number of 0's:", counts.get(0, 0))
print("Number of 1's:", counts.get(1, 0))



Number of 0's: 482
Number of 1's: 21368
